###### visualizing a neural machine Attention model(mechinics of seq2seq models with attention)

- 최근 10년 동안 자연어 처리 연구 중 가장 영향력이 컸던 3가지 중 하나가 바로 sequence-to-sequence + Attention 모델.

- 이후 수많은 연구들이 이런 encoder와 decoder가 결합된 seq2seq 모델 형태를 갖고 등장

- 서술했듯, Seq2seq는 기계번역, 문서 요약, 이미지 캡셔닝 등 다양한 NLP 태스크에서 큰 성공을 거둠. 
- 구글 번역기 역시 2016년 말부터 Seq2seq를 도입해서 활용 중. 

- encoder는 입력의 각 아이템을 처리하여 거기서 정보를 추출한 후 하나의 벡터를 만듬(흔히 context라고 부름)
- 입력의 모든 단어에 대한 처리가 끝나면 encoder는 context를 decoder에게 보내고 출력할 아이템이 하나씩 선택됨.

- 기계 번역의 경우 context가 하나의 vector 형태로 전달. 
- context는 float으로 이루어진 하나의 벡터. 
- 이 context vector의 크기는 모델을 처음 설정할 때 원하는 값으로 설정할 수 있음
- 보통은 encoder RNN의 hidden unit의 개수로 정함. (ex) 256, 512, 1024)


- seq2seq 모델 디자인을 보게 되면 하나의 RNN은 한 time step마다 두 개의 입력을 받음. 
- 하나는 sequence의 한 아이템이고, 다른 하나는 그전 스텝에서의 RNN의 hidden state임.
- 이 두 입력들은 RNN에 들어가기전 반드시 vector로 변환되어야 함. 
- 하나의 단어를 벡터로 바꾸기 위해서 word embedding이라는 기법을 활용. 
- word embedding을 통해 단어들은 벡터 공간에 투영되고, 그 공간에서 단어 간 다양한 의미와 관련 정보를 알아 낼 수 있음

- 앞서 설명한대로 encoder에서 단어들을 처리하기 전에 먼저 vector로 변환해야 함. 
- word embedding을 이용할 수도 있고, pre-trained embeddings를 이용하거나 데이터셋을 통해 직접 학습할 수 있음 
- 보통 200~ 300 의 embedding vector를 이용함.

- encoder와 decoder는 모두 RNN이며, RNN은 한 번 아이템을 처리할 때마다 새로들어온 아이템을 이용해 hidden state를 계속 업데이트 함.
- 이 hidden stste는 그에 따라 encoer가 보는 입력 시퀀스 내의 모든 단어 정보를 담게 됨. 
- 이때 마지막 단어의 hidden state가 바로 decoder에게 넘겨주는 context가 됨. 

- decoder 역시 decoder의 hidden states를 지니고 있으며, 스텝마다 업데이트를 함. 

<img src = "https://nlpinkorean.github.io/images/seq2seq/seq2seq_attention.png">

###### Attention
- 연구를 통해 context vector가 이런 seq2seq 모델의 가장 큰 걸림돌임이 밝혀짐. 
- 이렇게 하나의 고정된 vector로 전체의 맥락을 나타내는 방법은 긴 문장을 처리하기 어렵게 만듬. 
- 이에 대한 해결 방법으로 제시된 것이 바로 Attention 매커니즘.
- attention 메커니즘은 seq2seq 모델의 decoding 과정에서 현태 스텝에 가장 관련된 input part에 집중할 수 있도록 해줌으로써 기계 번역 품질을 향상시킴. 

- 참고 : https://nlpinkorean.github.io/images/seq2seq/seq2seq_7.mp4
- attention model과 기존의 seq2seq 모델은 2 가지의 차이점을 지님.
- 첫번째는 encoder가 decoder에 넘겨주는 데이터의 양이 attention 모델에서 훨씬 더 많음. 
- 기존의 seq2seq 모델에서 그저 마지막 아이템의 hidden state vector만 넘겼던 반면, attention model에서는 모든 step의 hidden states를 decoder에 넘김. 

- 참고 : https://nlpinkorean.github.io/images/seq2seq/seq2seq_attention_process.mp4
- 두 번째는 attention 모델의 decoder가 출력을 생성할 때 하나의 추가과정이 더 필요함. 
- decoder는 현재 스텝에서 관련있는 입력을 찾기 위한 과정을 실행
    + encoder에서 받은 전체 hidden state를 봄. 각 스텝에서의 encoder hidden states는 이전의 맥락에 대한 정보도 포함하고 있지만, 그 스텝에서의 입력 단어와 가장 관련이깊음.
    + 각 스텝의 hidden state 마다 점수를 매김
    + 매겨진 점수들에 softmax를 취하고 이것을 각 타임 스텝의 hidden states에 곱해서 더함. 이를 통해 높은 점수를 가진 hidden states는 더 큰 부분을 차지하게 되고 낮은 점수를 가진 hidden state는 작은 부분을 가져가게 됨. 

- 이렇게 점수를 매기는 과정이 dcoder가 단어를 생성하는 매 스텝마다 반복됨. 
1. attention 모델에서의 decoder RNN은 <END.>과 추가로 initial decoder hidden state를 입력받음
2. decoder RNN은 두 개의 입력을 가지고 새로운 hidden state vector를 출력. RNN의 출력 자체는 사용되지 않고 버려짐. 
3. Attention 과정 : encoder의 hidden state h4 벡터를 이용하여 그 스텝에 해당하는 context vector를 계산. 
4. h4와 C4를 하나의 벡터로 concatenate(연결, 이어쓰기) 함. 
5. 이 벡터를 feedforward 신경망(seq2seq 모델 내에서 함께 학습되는 layer)에 통과시킴.
6. feedforward 신경망에서 나오는 출력은 현재 타임 스텝의 출력 단어를 나타냄. 